<a href="https://colab.research.google.com/github/federicozanini/NLP/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A


In [ ]:
git clone https://github.com/YOUR-USERNAME/YOUR-REPOSITORY

In [ ]:
import os
import numpy as np
import pandas as pd
import json

def load_data(path):

    # DATAFRAME ROWS
    dataframerows = []

    with open(path) as f:
        data = json.load(f)

        for el in data['data']:
            title = el['title']
            paragraphs = el['paragraphs']

            for context_qas in paragraphs:
                  context = context_qas['context']
                  qas = context_qas['qas']

                  for a_q in qas: #[0]["answers"][0]["text"]
                      answer = a_q['answers']
                      question = a_q['question']
                      #id = a_q['id']

                      for a in answer:
                          text = a['text']
                          #answer_start = int(a['answer_start'])
                          #answer_end = answer_start+len(text)-1

                          # CREATE ROW
                          row = {
                              "title" : title,
                              #"paragraphs" : paragraphs,
                              #"context_qas" : context_qas,
                              "context" : context,
                              #"qas" : qas,
                              "id" : id,
                              "question" : question,
                              #"answer" : answer,
                              "answer_text" : text,
                              #"answer_start" : answer_start,
                              #"answer_end":answer_end
                              }

                          # APPEND ROW TO DATAFRAME ROWS
                          dataframerows.append(row)

        df = pd.DataFrame(dataframerows)
        df = df[["title", "context", "question", "answer_text"]] ##, "answer_start", "answer_end"]]

        return df

In [ ]:
path = 'SQuAD/Dataset/training_set.json'
df = load_data(path)
df.head(100)

In [ ]:
!pip install transformers

In [ ]:
## using bert tokenizer to encode data 
import tensorflow as tf
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
#cella per analizzare tokenizer.encode
input_ids = tokenizer.encode("my name is matteo 105, luca!1 cazzo")

print(input_ids)

tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

In [ ]:
# little utils func, find the index of a sequence inside a list 
def find_sublist(l1, l2):
  for i, el in enumerate(l1):
    if l1[i:i+len(l2)]==l2:
      return i,i+len(l2)-1   
  return -1

In [ ]:
'''

DATA GENERATOR, objective is to suply the net with batch belonging to the same Context

'''
import random 

def data_loader(df):
  title = []
  while True:   
    if title == []:
      title = list(df['title'].unique())    
    
    current = title.pop(random.randrange(len(title)))
    df0 = df[df['title']==current]
    
    encoded = []
    max_len = 0
    for i, t in df0.iterrows():
      encoded.append(tokenizer.encode(t[2],t[1]))
    
    #padding input sequence 
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(encoded, padding="post")

    X = np.array(padded_inputs)

    Y_start = np.zeros(X.shape, dtype='int32')
    Y_end  = np.zeros(X.shape, dtype='int32')

    for i, t in enumerate(df0['answer_text']):
      enc_ans = tokenizer.encode(t,add_special_tokens= False) 
      try:
        start, end = find_sublist(X[i].tolist(),enc_ans)
      except:
        print(len(X[i].tolist()))
        print(type(X[i].tolist()))
        print(X[i].tolist())
      Y_start[i,start]=1
      Y_start[i,end]=1

    yield(X,Y_start,Y_end)  

In [ ]:
X,Y_start,Y_end = next(data_loader(df))


In [ ]:
help(tokenizer.encode)

In [ ]:
import tensorflow as tf

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)